# **Tokenize, Lemmatize, Bag of Word**

Tokenize: `lower + lemmatize -> exclude punctuation characters -> bag of word`

Example:
`"how are you?" -> ["how", "are", "you", "?"] -> ["how", "are", "you"] -> [0, 0, 1, 1, 0, 1]`

using ***NLTK*** to tokenize and lemmatize the words.

In [ ]:
import nltk
import numpy as np

nltk.download('punkt') # downloads the punkt_tab for tokenize
nltk.download('wordnet') # downloads the wordnet for lemmatize

from nltk.stem import WordNetLemmatizer as wnl

def tokenize(sentence: str):
  return nltk.word_tokenize(sentence)

def lemmatize(word: str):
  return wnl().lemmatize(word.lower())

def bag_of_word(sentence: list[str], all_words: list[str]):
  '''
  return a List of booleans of size all_words
  1 if word of sentence existsing in all_words
  possition of the 1 will depend on the index of the word in all_words

  Example
      sentent = ["how", "are", "you"]
      all_words = ["hello", "world", "how", "are", "my", "friend", "you"]
      bag =       [0,        0,       1,     1,     0,    0,        1 ]
  '''
  lemmatized_sentence = [lemmatize(w) for w in sentence] # Lemmatize the input sentence
  bag = np.zeros(len(all_words), dtype=np.float32) # creating a numpy of length all_words

  '''
    enumarat example

    ["hey", "there"] -> [(0, "hey"), (1, "there")]
  '''

  for index, word in enumerate(all_words): # Iterate through all_words
    if word in lemmatized_sentence: # Check if the word from all_words is in the lemmatized input sentence
      bag[index] = 1.0

  return bag

# **Neural Network Model**

Feed forward neural network

In [ ]:
import torch.nn as nn

class NeuralNetwork(nn.Module):
  '''
    input_size: size of the input
    hidden_size: size of the output of the layer
    num_classes: number of classes
  '''
  def __init__(self, input_size, hidden_size, num_classes):
    super(NeuralNetwork, self).__init__()
    self.l1 = nn.Linear(input_size, hidden_size)
    self.l2 = nn.Linear(hidden_size, hidden_size)
    self.l3 = nn.Linear(hidden_size, num_classes)
    self.relu = nn.ReLU()

  def forward(self, x):
    out = self.l1(x)
    out = self.relu(out)

    out = self.l2(out)
    out = self.relu(out)

    out = self.l3(out)

    return out


# **Trainer**


In [ ]:
import json
import numpy as np

with open("intents.json", "r") as data:
  intents = json.load(data)

all_words = []
tags = []
xy = []

for intent in intents['intents']:
  tag = intent['tag']
  tags.append(tag)

  for patten in intent['patterns']:
    words = tokenize(patten)
    all_words.extend(words)
    xy.append((words, tag))

ignore_words = ["?", ".", ",", ">", "<", "!", ":", ";", "'", '"', "\\", "/", "{", "}", "[", "]"]
all_words = [lemmatize(w) for w in all_words if w not in ignore_words]
all_words = sorted(set(all_words)) # Sorts the list and removes any duplicate words using SET properties.
tags = sorted(set(tags)) # Sorts and removes duplicate words.

X_train = [] # contains the bag of words ex [0, 1, 1, 0, 0, 0, 1]
Y_train = [] # contains the index of tags

for (words, tag) in xy:
  bag = bag_of_word(words, all_words) # Pass all_words to bag_of_word
  X_train.append(bag) # Append the bag as a list

  label = tags.index(tag)
  Y_train.append(label) # Append the label

X_train = np.array(X_train)
Y_train = np.array(Y_train)

# **Data Set**


In [ ]:
import torch
import torch.nn as nn

from torch.utils.data import Dataset, DataLoader

class ChatDataSet(Dataset):
  def __init__(self):
    self.n_samples = len(X_train) # Number of samples
    self.x_data = X_train
    self.y_data = Y_train

  def __getitem__(self, index):
    return self.x_data[index], self.y_data[index]

  def __len__(self):
    return self.n_samples

batch_size = 8
hidden_size = 16 # Increased hidden size
output_size = len(tags)
input_size = len(all_words)
learning_rate = 0.0005 # Adjusted learning rate
num_epochs = 2000

data_set = ChatDataSet()
train_loader = DataLoader(dataset=data_set, batch_size=batch_size, shuffle=True)

# Check for GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

model = NeuralNetwork(input_size, hidden_size, output_size).to(device) # Move model to GPU

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
  for (words, labels) in train_loader:
    words = words.to(device) # Move data to GPU
    labels = labels.to(device) # Move labels to GPU

    # Forwards
    outputs = model(words)
    loss = criterion(outputs, labels)

    #Backward and optimizer step
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 100 == 0: # Corrected the print condition to print every 100 epochs
      print(f"epoch {epoch+1}/{num_epochs}, loss={loss.item():.4f}")

print(f"Final loss={loss.item():.4f}")

data = {
    'model_state': model.state_dict(),
    'input_size': input_size,
    'output_size': output_size,
    'hidden_size': hidden_size,
    'all_words': all_words,
    'tags': tags
}

FILE = "intents_data.pth" # Renamed the file
torch.save(data, FILE)

print(f"Training complete. File saved to {FILE}")

# **Chat**

In [ ]:
import torch
import random
import json # Import json to load intents

FILE = "intents_data.pth" # Updated file name
data = torch.load(FILE)

tags = data["tags"]
all_words = data["all_words"]
input_size = data["input_size"]
model_state = data["model_state"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]

# Check for GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = NeuralNetwork(input_size, hidden_size, output_size).to(device) # Move model to GPU
model.load_state_dict(model_state)
model.eval() # Sets of eval mode

# Load intents data for responses
with open("intents.json", "r") as json_data:
    intents = json.load(json_data)

print("Press 'quit' to exit")
while True:
  sentence = input("")
  if sentence == "quit":
    break

  sentence = tokenize(sentence)
  X = bag_of_word(sentence, all_words)
  X = X.reshape(1, X.shape[0])
  X = torch.from_numpy(X).to(device) # Move input tensor to GPU

  output = model(X)

  _, predicted = torch.max(output, dim=1)
  tag = tags[predicted.item()]

  probs = torch.softmax(output, dim=1)
  prob = probs[0][predicted.item()]

  print(prob)

  if prob.item() > 0.75:
    for intent in intents["intents"]:
      if tag == intent["tag"]:
        print(random.choice(intent['responses']))
  else:
    print("I can not understand...")